<a href="https://colab.research.google.com/github/Montse1708/Practica3_Metricas/blob/main/TI_1_Practica3_JessicaMontserratMoralesEnrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Jessica Montserrat Morales Enrique  
**e-mail:** jessica.morales5556@alumnos.udg.mx

## MODULES

In [ ]:
import math
import numpy as np
import pandas as pd
from scipy.stats import cauchy
import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

###Brownian Motions (BM)

In [ ]:
######################################################################
# Brownian Motion Trajectory (BM)
######################################################################

def BM_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  #Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  #Concatenar
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i] + velocity.x, 'y_pos': BM_2d_df.y_pos[i] + velocity.y}])
    #Concatenar
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
  return BM_2d_df

###Correlated Random Walks (CRW)

In [ ]:
######################################################################
# Correlated Random Walks (CRW)
######################################################################

def CRW_2d(CRW_exponents=0.2, n_steps=1000, speed=5, s_pos=[0,0]):
    # Init velocity vector
    velocity = Vec2d(speed, 0)
    # Guardar las trayectorias en Pandas DataFrame
    CRW_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    # Concatenar
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps-1 ):
        # Rotación de la trayectoria
        turn_angle = wrapcauchy.rvs(CRW_exponents)
        velocity = velocity.rotated(turn_angle)

        temp_df= pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i] + velocity.x, 'y_pos': CRW_2d_df.y_pos[i] + velocity.y}])
        # Concatenar
        CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
    return CRW_2d_df

###Función auxiliar que sustituye a distance.euclidean

In [197]:
def distance_euclidean(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

###Función alternativa calcula los valores de la curva de path de una trayectoria.

In [190]:
# Define your function to compute path length for given trajectory
## start - Add your code here
#def distance_euclidean(x1, y1, x2, y2):
 #   return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def path_length (trajectory):
  n_steps = trajectory.shape[0]
  array_trajectory = [];
  for i in range(1, n_steps):
    array_trajectory.append(distance_euclidean(trajectory.iloc[i-1], trajectory.iloc[i]))
    path_length_df = pd.DataFrame(np.cumsum(array_trajectory), columns=['PL'])
  return path_length_df

## end - Add your code here

###Mean Squared Displacement (MSD)

In [198]:
# Define your function to compute Mean Squared Displacement for given trajectory
## start - Add your code here
def MSD(trajectory):
    n_steps = len(trajectory)
    msd_values = []

    for i in range(1, n_steps):
        total_distance = 0
        for j in range(n_steps - i):
            # Calcular la distancia euclidiana entre los puntos j y j+i en la trayectoria
            total_distance += distance_euclidean((trajectory.x_pos[j], trajectory.y_pos[j]), (trajectory.x_pos[j + i], trajectory.y_pos[j + i]))
        # Calcular el MSD para el paso de tiempo i y agregarlo a la lista de valores de MSD
        msd_i = (total_distance / (n_steps - i))**2
        msd_values.append(msd_i)

    # Crear un DataFrame de pandas con los valores de MSD calculados
    msd_df = pd.DataFrame({'x_pos': trajectory.x_pos[:-1], 'y_pos': trajectory.y_pos[:-1], 'z_pos': msd_values})
    return msd_df
## end - Add your code here

###Turning angle function (TA)

In [ ]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array(pos_b) - np.array(pos_a)
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array(pos_c) - np.array(pos_b)
    norm_bc = np.linalg.norm(vec_bc)

    dot_p = np.dot(vec_ab, vec_bc)

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab*norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

###Calcular los ángulos de giro

In [ ]:
def calculateAngles(trajectory):
    aux_ta = np.empty(shape=(0))
    for index in range(1, len(trajectory) - 1):
        pos_a = trajectory.iloc[index - 1]
        pos_b = trajectory.iloc[index]
        pos_c = trajectory.iloc[index + 1]
        theta = turning_angle(pos_a, pos_b, pos_c)
        aux_ta = np.append(aux_ta, theta)
    TA_df = pd.DataFrame(aux_ta, columns=['TA'])
    return TA_df

###Levy flight

In [ ]:
def levy_flight(Levy_exponent=0.9, n_steps=1000, speed=5, s_pos=[0, 0]):
    # Inicializar vector de velocidad
    velocity = Vec2d(speed, 0)
    # Inicializar DataFrame para guardar la trayectoria
    levy_flight_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    # Concatenar
    levy_flight_df = pd.concat([levy_flight_df, temp_df], ignore_index=True)

    for i in range(n_steps-1):
        # Calcular el ángulo de giro con una distribución de Lévy
        turn_angle = levy_stable.rvs(Levy_exponent, 0, loc=0, scale=1)
        # Rotar el vector de velocidad
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': levy_flight_df.x_pos[i] + velocity.x, 'y_pos': levy_flight_df.y_pos[i] + velocity.y}])
        # Concatenar
        levy_flight_df = pd.concat([levy_flight_df, temp_df], ignore_index=True)

    return levy_flight_df

###Step Lengths

In [ ]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
def step_lengths(trajectory):
    n_steps = trajectory.shape[0]
    # Calcula las longitudes de paso utilizando la función distance_euclidean
    step_lengths = np.array([distance_euclidean(trajectory.iloc[i], trajectory.iloc[i+1]) for i in range(n_steps - 1)])
    # Crea un DataFrame con los resultados
    sl_df = pd.DataFrame(step_lengths, columns=['SL'])
    return sl_df

## end - Add your code here

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [179]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [186]:
# Get Path length calling the function
# Add your code here function(BM_2d_df_3)
PL_BM_3 = path_length(BM_2d_df_3)

# Get Path length calling the function
# Add your code here function(BM_2d_df_6)
PL_BM_6 = path_length(BM_2d_df_6)

# Get Path length calling the function
# Add your code here function(CRW_2d_df_9)
PL_CRW_6 = path_length(CRW_2d_df_9)

In [187]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PL_BM_3.PL))+1,
    y = PL_BM_3.PL,
    name = 'path_length_BM_3',
    showlegend = True
))
## end - Add your code here

# Second trace BM2
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PL_BM_6.PL))+1,
    y = PL_BM_6.PL,
    line = dict(width = 6),
    name = 'path_length_BM_6',
    showlegend = True
))
## end - Add your code here

# Third trace CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PL_CRW_6.PL))+1,
    y = PL_CRW_6.PL,
    name = 'path_length_CRW_6',
    showlegend = True
))
## end - Add your code here

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [172]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [188]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [200]:
# Get Mean Squared Displacement calling the function
# Add your code here function(BM_2d_df_6)
MSD_BM = MSD(BM_2d_df_6)

# Get Mean Squared Displacement calling the function
# Add your code here function(BM_2d_df_6)
MSD_CRW = MSD(CRW_2d_df_9)

In [201]:
# Init figure
fig_MSD = go.Figure()

# first trace MSD_BM
## start - Add your code here

fig_MSD.add_trace(go.Scatter(
    x = MSD_BM.x_pos.index,
    y = MSD_BM.z_pos,
    name = 'MSD BM 6',
    showlegend = True
))
## end - Add your code here

# Second trace MSD_CRW
## start - Add your code here
fig_MSD.add_trace(go.Scatter(
    x = MSD_CRW.x_pos.index,
    y = MSD_CRW.z_pos,
    name = 'MSD CRW 6',
    showlegend = True
))
## end - Add your code here

fig_MSD.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [ ]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))

# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6[1:-1].iterrows():
    ## start - Add your code here
    pos_a = CRW_2d_df_6.iloc[index - 1]
    pos_b = CRW_2d_df_6.iloc[index]
    pos_c = CRW_2d_df_6.iloc[index + 1]
    theta = turning_angle(pos_a, pos_b, pos_c)
    aux_ta_CRW_2d_df_6 = np.append(aux_ta_CRW_2d_df_6, theta)
    ## end - Add your code here

# aux to store turning angles
aux_ta_CRW_2d_df_9 = np.empty(shape=(0))

## start - Add your code here
for index, row in CRW_2d_df_9[1:-1].iterrows():
    pos_a = CRW_2d_df_9.iloc[index - 1]
    pos_b = CRW_2d_df_9.iloc[index]
    pos_c = CRW_2d_df_9.iloc[index + 1]
    theta = turning_angle(pos_a, pos_b, pos_c)
    aux_ta_CRW_2d_df_9 = np.append(aux_ta_CRW_2d_df_9, theta)
## end - Add your code here

In [ ]:
# Define your function to compute Turning Angles for given trajectory

## start - Add your code here
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array(pos_b) - np.array(pos_a)
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array(pos_c) - np.array(pos_b)
    norm_bc = np.linalg.norm(vec_bc)

    dot_p = np.dot(vec_ab, vec_bc)

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab*norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta
## end - Add your code here

In [ ]:
# Get Turning Angles calling the function
# Add your code here function(CRW_2d_df_6)
ta_CRW_2d_df_6 = calculateAngles(CRW_2d_df_6)

# Get Turning Angles calling the function
# Add your code here function(CRW_2d_df_9)
ta_CRW_2d_df_9 = calculateAngles(CRW_2d_df_9)

In [ ]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_df_3 = go.Figure()

# Histogram turning angle CRW_2d_df_6
fig_met_df_3.add_trace(go.Histogram(
                 x=ta_CRW_2d_df_6.TA,
                 name = 'observed 0.6',
                 showlegend = True,
                 xbins=dict(size=0.03),
                 histnorm='probability density'
                 ))
## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(
                 x=ta_CRW_2d_df_9.TA,
                 name = 'observed 0.9',
                 showlegend = True,
                 xbins=dict(size=0.03),
                 histnorm='probability density'
                 ))
## end - Add your code here


# Add origin distributions
## start - Add your code here
aux_domain = np.linspace(-3, 3, 1000)

cauchy_6 = cauchy.fit(ta_CRW_2d_df_6.TA)
y1 = cauchy.pdf(aux_domain, * cauchy_6)

fig_met_df_3.add_trace(go.Scatter(
                 x=aux_domain,
                 y= y1,
                 name = 'cauchy_0.6',
                 showlegend = True,
                 line=dict(color='blue')
                 ))

cauchy_9 = cauchy.fit(ta_CRW_2d_df_9.TA)
y2 = cauchy.pdf(aux_domain, *cauchy_9)

fig_met_df_3.add_trace(go.Scatter(
                 x=aux_domain,
                 y= y2,
                 name = 'cauchy_0.9',
                 showlegend = True,
                 line=dict(color='red')
                 ))
## end - Add your code here


fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [ ]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
def step_lengths(trajectory):
    n_steps = trajectory.shape[0]
    # Calcula las longitudes de paso utilizando la función distance_euclidean
    step_lengths = np.array([distance_euclidean(trajectory.iloc[i], trajectory.iloc[i+1]) for i in range(n_steps - 1)])
    # Crea un DataFrame con los resultados
    sl_df = pd.DataFrame(step_lengths, columns=['SL'])
    return sl_df

## end - Add your code here

In [ ]:
# Get Step lengths calling the function
# Add your code here function(Levy_2d_df_1)
sl_Levy_2d_df_1 = step_lengths(Levy_2d_df_1)

# Get Step lengths calling the function
# Add your code here function(Levy_2d_df_7)
sl_Levy_2d_df_7 = step_lengths(Levy_2d_df_7)

In [ ]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
# Histogram turning angle CRW_2d_df_6
fig_met_df_4.add_trace(go.Histogram(
                 x=sl_Levy_2d_df_1.SL,
                 name = 'Observed_alpha=1.0_beta=1.0',
                 showlegend = True,
                 histnorm='probability density',
                 xbins=dict(size=2)
                 ))

# Histogram step-length Levy_2d_df_7
fig_met_df_4.add_trace(go.Histogram(
                 x=sl_Levy_2d_df_7.SL,
                 name = 'Observed_alpha=0.7_beta=1.0',
                 showlegend = True,
                 histnorm='probability density'
                 ))


# Add origin distributions
## start - Add your code here
# Ajustar la distribución de Lévy a los datos de sl_Levy_2d_df_7.SL
params = levy_stable.fit(np.array(sl_Levy_2d_df_7.SL))

# Calcular la PDF ajustada con los parámetros encontrados
x = np.linspace(0, 200, 1000)
y_fitted = levy_stable.pdf(x, *params)

# Agregar la PDF ajustada al gráfico
fig_met_df_4.add_trace(go.Scatter(
    x=x,
    y=y_fitted,
    mode='lines',
    name='Fitted Levy Distribution',
    line=dict(color='green', dash='dash')
))

# Actualizar el rango del eje y para que se ajuste a las distribuciones
y_max = max(max(y1), max(y2), max(y_fitted))  # Obtener el máximo de las distribuciones
fig_met_df_4.update_yaxes(range=[0, y_max*1.1])  # Ajustar el rango del eje y

## end - Add your code here

fig_met_df_4.show()


KeyboardInterrupt: 

In [ ]:
# Ajustar distribuciones de Lévy a los datos de las trayectorias
params1 = levy_stable.fit(np.array(sl_Levy_2d_df_1.SL))
params2= levy_stable.fit(np.array(sl_Levy_2d_df_7.SL))

# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
# Histogram turning angle CRW_2d_df_6
fig_met_df_4.add_trace(go.Histogram(
                 x=sl_Levy_2d_df_1.SL,
                 name='Observed_alpha=1.0_beta=1.0',
                 showlegend=True,
                 histnorm='probability density',
                 xbins=dict(size=2)
                 ))

# Histogram step-length Levy_2d_df_7
fig_met_df_4.add_trace(go.Histogram(
                 x=sl_Levy_2d_df_7.SL,
                 name='Observed_alpha=0.7_beta=1.0',
                 showlegend=True,
                 histnorm='probability density'
                 ))

# Add origin distributions
x = np.linspace(0, 200, 1000)
fig_met_df_4.add_trace(go.Scatter(
                 x=x,
                 y=levy_stable.pdf(x, *params1),
                 name='Levy_alpha=1.0',
                 showlegend=True,
                 line=dict(color='blue')
                 ))

fig_met_df_4.add_trace(go.Scatter(
                 x=x,
                 y=levy_stable.pdf(x, *params2),
                 name='Levy_alpha=0.7',
                 showlegend=True,
                 line=dict(color='red')
                 ))

fig_met_df_4.update_layout(xaxis=dict(range=[0, 200]))

fig_met_df_4.show()

KeyboardInterrupt: 